
# 03. CatBoost 모델링 및 앙상블 (CatBoost & Soft Voting)

본 노트북은 **CatBoost Classifier**를 활용하여 CTR 예측 모델을 구축하는 과정을 다룹니다.
범주형 변수 처리에 강점이 있는 CatBoost를 기반으로, 시드(Seed) 앙상블을 통해 모델의 일반화 성능을 극대화합니다.

### 🛠️ 모델링 파이프라인
1. **Down-Sampling**: 클래스 불균형 해소를 위한 1:1 비율 샘플링 적용
2. **Feature Engineering**:
    - **Sequence Features**: 사용자 행동 이력(`seq`)을 활용한 길이, 빈도, 패턴 변수 생성
    - **Interaction Features**: 주요 범주형 변수 간의 조합(Interaction) 파생 변수
    - **Group Statistics**: ID별 상세 통계량 집계
3. **Model Training**:
    - `TossMetric` (LogLoss와 AP의 가중 결합)을 기반으로 한 학습 최적화
    - 3가지 Random Seed(42, 106, 1031)를 사용한 독립적 모델 학습
4. **Ensemble**: 학습된 모델 간의 Soft Voting을 통한 최종 확률값 산출


In [1]:

import pandas as pd
import numpy as np
import os
import random
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, log_loss, accuracy_score
from tqdm import tqdm
from itertools import combinations
import gc
import warnings

warnings.filterwarnings('ignore')

# CatBoost 설치 확인 (필요시)
try:
    import catboost
except ImportError:
    !pip install catboost


In [2]:

class CFG:
    # 데이터 경로 (사용자 환경에 맞게 수정)
    # 현재 위치: /models/ 이므로 데이터는 ../data/ 에 존재
    DATA_PATH = '../rawdata'
    
    # 💥 학습할 시드 리스트 (앙상블 효과)
    SEEDS = [42, 106, 1031]
    
    VALID_RATIO = 0.2
    
    # 생성할 파일명 Prefix
    OUTPUT_PREFIX = '03_CatBoost'

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

print(f"Data Path: {CFG.DATA_PATH}")


Data Path: ../rawdata


In [3]:

# --- 1. 클릭 확률 맵 로드 ---
try:
    df_click_prob = pd.read_excel(os.path.join(CFG.DATA_PATH, 'high_click_numbers.xlsx'))
    click_prob_map = dict(zip(df_click_prob['number'], df_click_prob['click_prob']))
except FileNotFoundError:
    print("⚠️ 'high_click_numbers.xlsx' not found. Skipping click_prob features.")
    click_prob_map = {}

# --- 2. 부정/긍정 리스트 ---
pos_list = {370, 528, 68, 561, 144, 227, 417, 442, 186, 395}
neg_list = {154, 222, 84, 498, 434, 511, 216, 497, 309, 446}

def add_seq_features(df, name="dataset"):
    # seq_len, avg_prob, seq_neg, seq_pos 리스트 생성
    seq_len, avg_prob, seq_neg, seq_pos = [], [], [], []

    for s in tqdm(df["seq"], desc=f"Processing {name}"):
        if isinstance(s, str) and s != "":
            arr = [int(x) for x in s.split(",") if x]

            # 길이
            seq_len.append(len(arr))

            # 클릭 확률 평균
            probs = [click_prob_map.get(num) for num in arr if num in click_prob_map]
            avg_prob.append(sum(probs) / len(probs) if probs else np.nan)

            # neg/pos 카운트
            seq_neg.append(sum(1 for x in arr if x in neg_list))
            seq_pos.append(sum(1 for x in arr if x in pos_list))
        else:
            seq_len.append(0)
            avg_prob.append(np.nan)
            seq_neg.append(0)
            seq_pos.append(0)

    df["seq_len"] = seq_len
    df["avg_click_prob"] = avg_prob
    df["seq_neglogcount"] = seq_neg
    df["seq_poslogcount"] = seq_pos

    # avg_click_prob 결측치 처리
    overall_avg_prob = df["avg_click_prob"].mean()
    df["avg_click_prob"].fillna(overall_avg_prob, inplace=True)

    return df

def add_groupby_features(train_df, test_df, feature_name, agg_dict):
    print(f"--- Creating features by grouping '{feature_name}' ---")
    agg_stats = train_df.groupby(feature_name).agg(agg_dict)
    new_cols = [f"{feature_name}_{col[0]}_{col[1]}" for col in agg_stats.columns]
    agg_stats.columns = new_cols
    agg_stats.reset_index(inplace=True)

    train_df = pd.merge(train_df, agg_stats, on=feature_name, how='left')
    test_df = pd.merge(test_df, agg_stats, on=feature_name, how='left')

    for col in new_cols:
        fill_value = train_df[col].mean()
        test_df[col].fillna(fill_value, inplace=True)
    
    return train_df, test_df


def add_count_features(train_df, test_df, count_cols):
    print("--- Creating Count Encoding Features ---")
    for col in count_cols:
        # Use concat to get global counts
        all_data = pd.concat([train_df[col], test_df[col]], ignore_index=True)
        count_map = all_data.value_counts().to_dict()
        
        train_df[f"{col}_count"] = train_df[col].map(count_map).fillna(0).astype(int)
        test_df[f"{col}_count"] = test_df[col].map(count_map).fillna(0).astype(int)
    return train_df, test_df


In [4]:

class TossMetric:
    def is_max_optimal(self): return True
    def evaluate(self, approxes, target, weight):
        y_pred = 1.0 / (1.0 + np.exp(-approxes[0]))
        y_true = np.array(target)
        # Check imbalance
        N_1, N_0 = np.sum(y_true), len(y_true) - np.sum(y_true)
        if N_1 == 0 or N_0 == 0: return 0.5, 0
        
        # Weighted LogLoss
        weights = np.where(y_true == 1, 0.5 / N_1, 0.5 / N_0)
        y_pred_clipped = np.clip(y_pred, 1e-15, 1 - 1e-15)
        wll = log_loss(y_true, y_pred_clipped, sample_weight=weights)
        
        # AP
        ap = average_precision_score(y_true, y_pred)
        
        # Score Formula
        score = 0.5 * ap + 0.5 * (1 / (1 + wll))
        return score, 1.0
        
    def get_final_error(self, scores, weight): return np.mean(scores)


In [5]:

# --- Main Logic Loop ---

# 결과 파일들을 저장할 리스트
prediction_files = []

# 피처 엔지니어링에 사용할 Aggregation 정의
# 피처 엔지니어링에 사용할 Aggregation 정의 (Original Logic 추가)
aggs_to_create = {
    'history_a_1': ['mean', 'std'], 'history_a_2': ['mean', 'std'], 
    'history_a_3': ['mean', 'std'], 'history_a_6': ['mean', 'std'],
    'feat_e_2': ['mean', 'std'], 'feat_e_3': ['mean', 'std'],
    'feat_c_8' : ['mean', 'std'], 'feat_e_9' : ['mean', 'std'],
    'feat_d_4' : ['mean', 'std'], 
    'l_feat_1' : ['mean', 'std'], 'l_feat_2' : ['mean', 'std'],
    'l_feat_5' : ['mean', 'std'], 'l_feat_7' : ['mean', 'std'],
    'l_feat_10' : ['mean', 'std'], 'l_feat_15' : ['mean', 'std']
}

for seed in CFG.SEEDS:
    print(f"\n{'='*40}\n🚀 Processing Seed: {seed}\n{'='*40}")
    seed_everything(seed)
    
    # 1. 데이터 로드 (매 반복마다 원본 로드)
    # 10% 샘플링된 데이터 사용 (전체 데이터 사용 시 파일명을 'train.parquet'으로 변경)
    train_path = os.path.join(CFG.DATA_PATH, 'D:/folder\대학원\CTR/rawdata/train_sample_10pct.parquet') 
    train_df = pd.read_parquet(train_path)
    # 테스트 데이터 로드 (없으면 더미 생성 로직 필요하지만 여기서는 있다고 가정)
    try:
        test_df = pd.read_parquet(os.path.join(CFG.DATA_PATH, 'test.parquet'))
    except FileNotFoundError:
        print("⚠️ Test data not found. Creating dummy test data for demo.")
        # 더미 데이터 생성 (실제 환경에서는 제거)
        test_df = train_df.head(100).drop(columns=['clicked'])
        test_df['ID'] = [f'TEST_{i}' for i in range(100)] 
    
    # 2. Down-Sampling (1:1 Ratio)
    print("🔻 Down-sampling...")
    clicked_1 = train_df[train_df['clicked'] == 1]
    clicked_0 = train_df[train_df['clicked'] == 0].sample(n=int(len(clicked_1)), random_state=seed)
    train_df = pd.concat([clicked_1, clicked_0], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
    print(f"   Train shape: {train_df.shape}")
    
    # 3. Feature Engineering
    print("🛠️ Feature Engineering...")
    
    # Seq Features
    train_df = add_seq_features(train_df, "train")
    test_df = add_seq_features(test_df, "test")
    
    # Interaction Features
    cols = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']
    for col1, col2 in combinations(cols, 2):
        new_col = f'{col1}_{col2}'
        train_df[new_col] = train_df[col1].astype(str) + '_' + train_df[col2].astype(str)
        test_df[new_col] = test_df[col1].astype(str) + '_' + test_df[col2].astype(str)
        train_df[new_col] = train_df[new_col].astype('category')
        test_df[new_col] = test_df[new_col].astype('category')
        
    # Groupby Features
    train_df, test_df = add_groupby_features(train_df, test_df, 'inventory_id', aggs_to_create)
    
    
    # Count Encoding
    count_cols = ['age_group', 'inventory_id', 'day_of_week'] # 주요 변수 카운트
    # 'age_group_inventory_id', 'inventory_id_hour' 등 조합 변수가 있다면 추가
    train_df, test_df = add_count_features(train_df, test_df, count_cols)

    # Fill NA & Clean
    categorical_features = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour', 'gender_age_group',
                            'gender_inventory_id', 'gender_day_of_week', 'gender_hour', 'age_group_inventory_id',
                            'age_group_day_of_week', 'age_group_hour', 'inventory_id_day_of_week', 'inventory_id_hour',
                            'day_of_week_hour']
                            
    for col in categorical_features:
        if col in train_df.columns:
            train_df[col] = train_df[col].astype(str).fillna('NaN').astype('category')
            test_df[col] = test_df[col].astype(str).fillna('NaN').astype('category')
            
    for df in [train_df, test_df]:
        for col in df.columns:
            if pd.api.types.is_numeric_dtype(df[col]):
                df[col] = df[col].fillna(0)
    
    # 4. Training
    print("🔥 Training CatBoost...")
    target_col = "clicked"
    exclude_cols = {target_col, "seq", "ID"}
    features = [c for c in train_df.columns if c not in exclude_cols]
    cat_features = [c for c in features if c in categorical_features] # Only use valid cat features
    
    X_train, X_val, y_train, y_val = train_test_split(
        train_df[features], train_df[target_col], 
        test_size=CFG.VALID_RATIO, random_state=seed, stratify=train_df[target_col]
    )
    
    model = cb.CatBoostClassifier(
        iterations=1000, # 데모용 줄임, 실제는 10000
        learning_rate=0.02,
        depth=8,
        random_seed=seed,
        thread_count=-1,
        eval_metric=TossMetric(),
        early_stopping_rounds=100,
        verbose=100
    )
    
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], cat_features=cat_features)
    
    # 5. Prediction
    print("🔮 Predicting...")
    # predict_proba for class 1
    preds = model.predict_proba(test_df[features])[:, 1]
    
    # 6. Save Individual Result
    output_filename = f"{CFG.OUTPUT_PREFIX}_{seed}.csv"
    
    # 테스트 데이터에 ID 컬럼이 있다고 가정 (없으면 인덱스 사용)
    if 'ID' in test_df.columns:
        sub = pd.DataFrame({'ID': test_df['ID'], 'clicked': preds})
    else:
        # 실제 submission 파일 로드하여 ID 매핑 필요 (생략시 인덱스)
        sub = pd.DataFrame({'ID': range(len(preds)), 'clicked': preds})
        
    sub.to_csv(output_filename, index=False)
    prediction_files.append(output_filename)
    print(f"✅ Saved result to {output_filename}")
    
    # 메모리 정리
    del train_df, X_train, X_val
    gc.collect()

print("\n🏁 All seeds processed!")



🚀 Processing Seed: 42
🔻 Down-sampling...
   Train shape: (40546, 119)
🛠️ Feature Engineering...


Processing test: 100%|██████████| 1527298/1527298 [02:58<00:00, 8551.67it/s]


--- Creating features by grouping 'inventory_id' ---
--- Creating Count Encoding Features ---
🔥 Training CatBoost...
0:	learn: 0.6414476	test: 0.6361165	best: 0.6361165 (0)	total: 736ms	remaining: 12m 15s
100:	learn: 0.6798209	test: 0.6654646	best: 0.6654646 (100)	total: 14.3s	remaining: 2m 7s
200:	learn: 0.6940942	test: 0.6699310	best: 0.6699310 (200)	total: 28.1s	remaining: 1m 51s
300:	learn: 0.7049440	test: 0.6723240	best: 0.6723240 (300)	total: 42.3s	remaining: 1m 38s
400:	learn: 0.7140569	test: 0.6738242	best: 0.6738552 (398)	total: 55.9s	remaining: 1m 23s
500:	learn: 0.7228176	test: 0.6747322	best: 0.6747341 (498)	total: 1m 9s	remaining: 1m 8s
600:	learn: 0.7323062	test: 0.6753737	best: 0.6754183 (579)	total: 1m 22s	remaining: 54.5s
700:	learn: 0.7427383	test: 0.6760500	best: 0.6760764 (692)	total: 1m 35s	remaining: 40.6s
800:	learn: 0.7517399	test: 0.6762834	best: 0.6762834 (800)	total: 1m 48s	remaining: 27s
900:	learn: 0.7596509	test: 0.6767309	best: 0.6767940 (881)	total: 2m 1

Processing test: 100%|██████████| 1527298/1527298 [02:54<00:00, 8759.52it/s]


--- Creating features by grouping 'inventory_id' ---
--- Creating Count Encoding Features ---
🔥 Training CatBoost...
0:	learn: 0.6255548	test: 0.6242829	best: 0.6242829 (0)	total: 189ms	remaining: 3m 8s
100:	learn: 0.6796333	test: 0.6689177	best: 0.6689177 (100)	total: 12.7s	remaining: 1m 53s
200:	learn: 0.6936070	test: 0.6735137	best: 0.6735137 (200)	total: 25.2s	remaining: 1m 40s
300:	learn: 0.7038193	test: 0.6755912	best: 0.6755928 (299)	total: 37.8s	remaining: 1m 27s
400:	learn: 0.7123894	test: 0.6769260	best: 0.6769294 (399)	total: 50.2s	remaining: 1m 14s
500:	learn: 0.7209331	test: 0.6779858	best: 0.6779858 (500)	total: 1m 2s	remaining: 1m 2s
600:	learn: 0.7300700	test: 0.6785162	best: 0.6785982 (589)	total: 1m 15s	remaining: 50s
700:	learn: 0.7393660	test: 0.6790900	best: 0.6791115 (691)	total: 1m 27s	remaining: 37.4s
800:	learn: 0.7485119	test: 0.6789778	best: 0.6792279 (777)	total: 1m 40s	remaining: 24.9s
900:	learn: 0.7560808	test: 0.6792540	best: 0.6792614 (899)	total: 1m 52

Processing test: 100%|██████████| 1527298/1527298 [02:53<00:00, 8813.98it/s]


--- Creating features by grouping 'inventory_id' ---
--- Creating Count Encoding Features ---
🔥 Training CatBoost...
0:	learn: 0.6373777	test: 0.6296628	best: 0.6296628 (0)	total: 158ms	remaining: 2m 37s
100:	learn: 0.6818855	test: 0.6625761	best: 0.6625761 (100)	total: 12.7s	remaining: 1m 52s
200:	learn: 0.6956936	test: 0.6661410	best: 0.6661410 (200)	total: 25.5s	remaining: 1m 41s
300:	learn: 0.7057635	test: 0.6677409	best: 0.6677409 (300)	total: 38.3s	remaining: 1m 28s
400:	learn: 0.7145724	test: 0.6690983	best: 0.6690983 (400)	total: 51.2s	remaining: 1m 16s
500:	learn: 0.7225827	test: 0.6697614	best: 0.6697933 (497)	total: 1m 3s	remaining: 1m 3s
600:	learn: 0.7313299	test: 0.6701476	best: 0.6701727 (575)	total: 1m 16s	remaining: 50.7s
700:	learn: 0.7406551	test: 0.6707380	best: 0.6707380 (700)	total: 1m 28s	remaining: 37.9s
800:	learn: 0.7490444	test: 0.6710650	best: 0.6710650 (800)	total: 1m 41s	remaining: 25.2s
900:	learn: 0.7567595	test: 0.6712584	best: 0.6712898 (899)	total: 1m

In [6]:

# --- Soft Voting Ensemble ---
print(f"\n🤝 Performing Soft Voting on {len(prediction_files)} files: {prediction_files}")

if len(prediction_files) > 0:
    merged = pd.read_csv(prediction_files[0]).rename(columns={"clicked": f"click_{CFG.SEEDS[0]}"})
    
    for i, file in enumerate(prediction_files[1:], start=1):
        df_temp = pd.read_csv(file).rename(columns={"clicked": f"click_{CFG.SEEDS[i]}"})
        merged = merged.merge(df_temp, on="ID")
    
    # Mean
    click_cols = [c for c in merged.columns if c.startswith("click_")]
    merged["clicked"] = merged[click_cols].mean(axis=1)
    
    # Save Final
    final_output = "03_CatBoost_SoftVoting.csv"
    merged[["ID", "clicked"]].to_csv(final_output, index=False)
    print(f"🎉 Final CatBoost Ensemble Saved: {final_output}")
    print(merged.head())
else:
    print("❌ No prediction files found.")



🤝 Performing Soft Voting on 3 files: ['03_CatBoost_42.csv', '03_CatBoost_106.csv', '03_CatBoost_1031.csv']
🎉 Final CatBoost Ensemble Saved: 03_CatBoost_SoftVoting.csv
             ID  click_42  click_106  click_1031   clicked
0  TEST_0000000  0.465993   0.371046    0.403368  0.413469
1  TEST_0000001  0.342321   0.350810    0.346905  0.346678
2  TEST_0000002  0.410186   0.440518    0.443540  0.431415
3  TEST_0000003  0.480882   0.466825    0.488084  0.478597
4  TEST_0000004  0.369274   0.378560    0.423092  0.390308
